# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [3]:
from  agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

In [4]:
nutrition_agent = Agent(name="nutrition-agent", instructions="You are a nutrition expert. You give concise answers based on scientific evidence.")

In [5]:
with trace("Simple Nutrition Agent") as t:
    result = await Runner().run(nutrition_agent, "What are the health benefits of eating apples?")
    print(result)

RunResult:
- Last agent: Agent(name="nutrition-agent", ...)
- Final output (str):
    - Rich in fiber (especially pectin): supports digestion, helps fullness, may lower LDL cholesterol.
    - Vitamins and minerals: good source of vitamin C and potassium; provides other micronutrients.
    - Polyphenols (e.g., quercetin): may reduce inflammation and support heart health; some anticancer and antioxidant tests are preliminary.
    - Heart and metabolic benefits: observational data link higher apple intake with lower risk of stroke, heart disease, and type 2 diabetes; fiber and polyphenols likely contribute.
    - Weight management: fiber and water content promote fullness with relatively few calories.
    - Gut microbiome: pectin and other compounds may improve gut bacterial balance.
    - Practical note: most benefits come from eating whole apples with skin; juice or dried forms have less fiber and more concentrated sugars.
    
    If you’d like, I can tailor to your health goals (heart

In [6]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="nutrition-agent", ...)
- Final output (str):
    Overall, bananas are a healthy, convenient fruit when eaten as part of a balanced diet.
    
    Key benefits:
    - Nutrients: good source of potassium (supports blood pressure and heart function), vitamin B6, vitamin C, and dietary fiber.
    - Low in fat and calories (about 100–105 kcal for a medium banana).
    - Fiber and resistant starch (especially when less ripe) support gut health and satiety.
    - Convenient, portable, and affordable.
    
    Considerations:
    - They contain natural sugars, so portion size matters for those watching blood sugar (diabetics should account for carbs).
    - People with kidney disease may need to monitor potassium intake.
    - Unripe bananas have more resistant starch; ripe bananas are sweeter with more sugar.
    
    Practical tips:
    - Pair with protein or fats (e.g., peanut butter, yogurt) for staying power.
    - One medium banana is a typical servin

Streaming the answer to the screen, token by token

In [7]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are a healthy, nutrient-dense fruit when eaten as part of a balanced diet.

Key points:
- Nutrients: good source of potassium, vitamin B6, vitamin C, and dietary fiber.
- Carbs and sugar: provide quick energy; about 14 g sugar per medium banana.
- Fiber: helps with satiety and gut health; resistant starch is higher when unripe.
- Calories: about 105 per medium banana; low in fat and protein.
- Health associations: linked to better blood pressure control and gut health; may aid athletic performance due to carbohydrate content.

Considerations:
- People with kidney disease or potassium-sensitive conditions may need to limit intake.
- Green (unripe) bananas have more resistant starch; ripe bananas are sweeter and digest faster.
- Pair with protein or fat for better satiety and blood sugar balance.

Bottom line: bananas are a healthy, convenient fruit choice for most people in moderation.

_Good Job!_